# Luna Training Dashboard

__This notebook is meant to setup training sessions, monitor training and evaluate its performances__
______

Load python libraries

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras.optimizers as optimizers
import keras.callbacks as callbacks
import keras.initializers as initializers

Using TensorFlow backend.


Load and setup local dependencies

In [2]:
cur_dir = os.getcwd()
base_dir = os.path.dirname(cur_dir)

import sys
sys.path.append(base_dir)
from utils.LungsLoader import LungsLoader
from utils.ScanHandler import ScanHandler

from src.networks.networks_utils import blocks
from src.networks.MariaNet import MariaNet
from src.training.config_file import ConfigFile
import src.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

loader = LungsLoader()
handler = ScanHandler(plt)

### Network hyperparameters

First hyperparameter showing up would be the input size which is here set to 256x256x256

In [3]:
input_shape = (256, 256, 256)

To build the network defined by [Christodoulis et al. 2018](https://arxiv.org/abs/1809.06226) we need to define :

- A sequence of encoding convolutional blocks which are to be concatenated. The original architecture consists of 5 dilated (conv + instanceNorm + LeakyRelu) block

    - Conv parameters :
        - kernel size : 3x3
        - Number filters : 32 - 64 - 128 - 32 - 32
        - Dilation rate : 1 - 1 - 2 - 3 - 5
    - LeakyRelu : `alpha` = 0.3
    
- A Squeeze Excitation block for the deformable decoder
- A sequence of decoding convolutional blocks for deformable registration. The original architecture consists of 5 non-dilated (conv + instanceNorm + LeakyRelu) blocks 
    - Conv parameters:
        - kernel size : 3x3
        - Number filters : 128 - 64 - 32 - 32 - 32
    - LeakyRelu : `alpha` = 0.3

Definition of the core blocks

In [4]:
conv_block_kwgs = {
                    "activation": "LeakyReLU",
                    "activation_kwargs":{
                        "alpha": 0.3
                        },
                    "normalize": True,
                    "conv_kwargs": {
                      "kernel_size": 3,
                      "padding": "same",
                      "kernel_initializer": initializers.RandomNormal(mean=0.0, stddev=1e-5)
                      }
                    }
squeeze_ratio = 16
conv_block = blocks.ConvBlock(**conv_block_kwgs)
squeeze_block = blocks.SqueezeExciteBlock(ratio=squeeze_ratio)

Encoding and decoding sequences of hyperparameters

In [5]:
enc_filters = [32, 64, 128, 32, 32]
enc_dilation = [(1, 1, 1), (1, 1, 1), (2, 2, 2), (3, 3, 3), (5, 5, 5)]
enc_params = [{"filters": n_filter, "dilation_rate": dil_rate} for (n_filter, dil_rate) in zip(enc_filters, enc_dilation)]

dec_filters = [128, 64, 32, 32, 32]
dec_params = [{"filters": n_filter} for n_filter in dec_filters]

Finally, the original paper adds an ultimate regularized convolution blocks at the end of the linear and deformable decoders. The deformable one consists of a (conv + sigmoid) block with 3 filters and the linear one of a (conv + linear) block with 12 filters.

In [6]:
def_flow_nf = 3
lin_flow_nf = 12

The model can now be generated

In [7]:
marianet = MariaNet(input_shape,
                    enc_params,
                     dec_params,
                     conv_block,
                     squeeze_block,
                     def_flow_nf,
                     lin_flow_nf)
model = marianet.build()
# model.summary()

### Training Session Setup